# Visualiation for the Experiment 1

In [1]:
%run exp1_tools.ipynb

In [2]:
import pandas as pd
import matplotlib.pylab as plt

/Applications/anaconda3/envs/qiskit-terra-dev/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Parameters

In [3]:
file_path = "~/Datastore/qiskit-terra/"
layout_methods = ['csplayout']

/Applications/anaconda3/envs/qiskit-terra-dev/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Collect Experiments

In [14]:
df = pd.DataFrame()
for lm in layout_methods:
    filename = file_path + "exp1_"+ str(lm) + ".csv"
    local_df = pd.read_csv(filename, index_col=[0])
    local_df = local_df.assign(method=[lm] * len(local_df))
    df = pd.concat([df, local_df])

/Applications/anaconda3/envs/qiskit-terra-dev/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
seeds = pd.unique(df["seed"])
seed_df_grps = {seed: df.loc[df["seed"] == seed].groupby("")}

In [15]:
df

/Applications/anaconda3/envs/qiskit-terra-dev/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,seed,state,ideal_count,noise_count,ideal_time,noise_time,ideal_swaps_needed,noise_swaps_needed,method
0,0,111111001011,2,0,0.004328,0.002708,0,0,csplayout
1,0,110110110,2,4,0.004328,0.002708,0,0,csplayout
2,0,1011000001,3,5,0.004328,0.002708,0,0,csplayout
3,0,1111011001,0,1,0.004328,0.002708,0,0,csplayout
4,0,111010010,2,2,0.004328,0.002708,0,0,csplayout
...,...,...,...,...,...,...,...,...,...
4014,0,101010100011,0,1,0.004328,0.002708,0,0,csplayout
4015,0,110100111000,2,4,0.004328,0.002708,0,0,csplayout
4016,0,111011100001,1,1,0.004328,0.002708,0,0,csplayout
4017,0,110111001011,2,2,0.004328,0.002708,0,0,csplayout
